# 타입B 태그안뽑기

In [2]:
import re
from tqdm.notebook import tnrange
import csv
import json
from os import walk
import os
import pandas as pd
import datetime

In [3]:
# 파일 불러오기
def get_csv(input_repo, input_filename):
    result = pd.read_csv(f'{input_repo}/{input_filename}')
    
    return result

In [4]:
# 전체 문장에서 태그 안만 뽑음
# <=법률어휘 ㅇㅇㄴㅁ
def save_tag(sentence):
    regex = re.compile('(?<=\<{3}).*?(?=\>{3})')
    result = regex.findall(sentence)
    
    return result

In [5]:
# 태그 내 문장으로부터 법률어휘 추출
def draw_word(sentence_tagremoved):
    regex = re.compile('(?<=\<=).*(?=\>)')
    result = regex.search(sentence_tagremoved)
    
    return result

In [6]:
def get_small_tag_removed(sentence):
    keyword_list = save_tag(sentence)
    regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
    small_tag_removed = []
    small_tag = []
    for i in range(len(keyword_list)):
        small_tag.append(regex_small_tag.findall(keyword_list[i]))
        small_tag_removed.append(keyword_list[i])

        for j in range(len(small_tag[i])):
            small_tag_removed[i] = small_tag_removed[i].replace(small_tag[i][j], '')
            
    return small_tag_removed

In [7]:
def get_small_tag_removed_throw_morethan2_org(sentence):
    keyword_list = save_tag(sentence)
    regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
    regex_small_tag_org = re.compile('(?<=\<={1}).*')

    small_tag_removed_org = []
    small_tag_list_org = []

    for i in range(len(keyword_list)):
        small_tag = regex_small_tag.findall(keyword_list[i])
        if len(small_tag)<2:
            try:
                small_tag_list_org = regex_small_tag_org.search(keyword_list[i]).group()
                small_tag_removed_org.append(small_tag_list_org.replace('>', ''))
            except:
                small_tag_removed_org.append(keyword_list[i])


        else:
            keyword_list[i] = None

    for i in range(len(keyword_list)-1, -1, -1):
        if keyword_list[i] == None:
            del keyword_list[i]

    return small_tag_removed_org

In [8]:
def get_small_tag_removed_throw_morethan2(sentence):
    keyword_list = save_tag(sentence)
    regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
    regex_small_tag_sub = re.compile('(?=\<={1}).*')
    small_tag_removed = []
    small_tag_list = []

    for i in range(len(keyword_list)):
        small_tag = regex_small_tag.findall(keyword_list[i])
        small_tag_sub = regex_small_tag_sub.findall(keyword_list[i])
        if len(small_tag)<2:
            try:           # 정상적인 경우
                small_tag_list.append(regex_small_tag.search(keyword_list[i]).group())
                small_tag_removed.append(keyword_list[i].replace(small_tag[0], ''))
            except:
                try:       # =>>>> 가 나올 때
                    small_tag_list.append(regex_small_tag_sub.search(keyword_list[i]).group())
                    small_tag_removed.append(keyword_list[i].replace(small_tag_sub[0], ''))
                except:    # <<<>>> 안에 <=> 아예 없을 때
                    small_tag_removed.append(keyword_list[i])

        else:
            keyword_list[i] = None

    for i in range(len(keyword_list)-1, -1, -1):
        if keyword_list[i] == None:
            del keyword_list[i]

    return small_tag_removed

In [9]:
# csv형태로 저장
def save_as_csv(input_repo, output_repo, result):
    year = str(datetime.datetime.now().year)[2:]
    month = str(datetime.datetime.now().month)
    day = str(datetime.datetime.now().day)
    hour = str(datetime.datetime.now().hour)
    minute = str(datetime.datetime.now().minute)

    print(f'{output_repo}의 경로에 최종_{year}{month}{day}_{hour}{minute}.csv의 이름으로 파일 생성이 완료되었습니다.')

    result.to_csv(f'{output_repo}/최종_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')

In [10]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기'
output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기/TypeB 최종'

input_filename = '태그안뽑기_211117_1618.csv'    # 바꿔

In [11]:
org = get_csv(input_repo, input_filename)

In [12]:
result_transcription_org = []
for i in tnrange(len(org)):
    result_temp = get_small_tag_removed_throw_morethan2_org(str(org['typeA'][i]))
    result_transcription_org.append(result_temp)

  0%|          | 0/76870 [00:00<?, ?it/s]

In [13]:
result_transcription = []
for i in tnrange(len(org)):
    result_temp = get_small_tag_removed_throw_morethan2(str(org['transcription'][i]))
    result_transcription.append(result_temp)

  0%|          | 0/76870 [00:00<?, ?it/s]

In [14]:
result = pd.DataFrame(columns=['법률 용어', '쓰임새', '번역예시', '키워드'])
result['쓰임새'] = []
result['번역예시'] = []
for i in tnrange(len(org)):
    word = org['word'][i]
    for j in range(len(result_transcription_org[i])):
        word_org = result_transcription_org[i][j]
        try:
            trans = result_transcription[i][j]
        except:
            trans = None
        keyword = None
#         if (word in word_org)&(word_org != word)&(trans!=None):
        if (word in word_org)&(trans!=None):
                result = result.append({'법률 용어':word, '쓰임새':word_org, '번역예시':trans, '키워드':keyword}, ignore_index=True)
        else:
            pass

  0%|          | 0/76870 [00:00<?, ?it/s]

In [15]:
result

,법률 용어,쓰임새,번역예시,키워드
0,호환,호환성을 갖는,연결되는 데 문제가 없는,None
1,호환,호환성이 있어,연결되는 데 문제가 없어,None
2,호환,호환컴퓨터에 대표적으로,연결되는 데 문제가 없는 컴퓨터에 대표적으로,None
3,호환,호환성을 가져야,연결되는 데 문제가 없기는,None
4,호환,호환될 것을,연결되는 데 문제가 없게될 것을,None
...,...,...,...,...
75466,종용,종용에 관한,잘 설득하고 달래어 권함에 관한,None
75467,종용,종용행위가 부당노동행위에,잘 설득하고 달래어 권하는행위가 부당노동행위에,None
75468,종용,종용한 것으로,잘 설득하고 달래어 권한 것으로,None
75469,종용,종용한 사실을,잘 설득하고 달래어 권한 사실을,None


In [18]:
len(set(result['법률 용어']))

2655

In [19]:
save_as_csv(input_repo, output_repo, result)

/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기/TypeB 최종의 경로에 최종_211122_1543.csv의 이름으로 파일 생성이 완료되었습니다.


In [18]:
# nan값 처리해야함. str로 넣어놨음